 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8890/notebooks/16-unet-inception.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8890/notebooks/16-unet-inception.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8890/notebooks/16-unet-inception.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8890/notebooks/16-unet-inception.ipynb#Modeling" data-toc-modified-id="Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="http://localhost:8890/notebooks/16-unet-inception.ipynb#v2" data-toc-modified-id="v2-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>v2</a></span></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import RMSprop, Adam
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Define loss functions

In [2]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [3]:
import sys
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Dense
from keras.layers import BatchNormalization, Dropout, Flatten, Lambda
from keras.layers.advanced_activations import ELU, LeakyReLU

IMG_ROWS, IMG_COLS = 128, 128

def _shortcut(_input, residual):
    stride_width = _input._keras_shape[2] / residual._keras_shape[2]
    stride_height = _input._keras_shape[3] / residual._keras_shape[3]
    equal_channels = residual._keras_shape[1] == _input._keras_shape[1]

    shortcut = _input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Convolution2D(nb_filter=residual._keras_shape[1], nb_row=1, nb_col=1,
                                 subsample=(stride_width, stride_height),
                                 init="he_normal", border_mode="valid")(_input)

    return merge([shortcut, residual], mode="sum")


def inception_block(inputs, depth, batch_mode=0, splitted=False, activation='relu'):
    assert depth % 16 == 0
    actv = activation == 'relu' and (lambda: LeakyReLU(0.0)) or activation == 'elu' and (lambda: ELU(1.0)) or None
    
    c1_1 = Convolution2D(int(depth/4), 1, 1, init='he_normal', border_mode='same')(inputs)
    
    c2_1 = Convolution2D(int(depth/8*3), 1, 1, init='he_normal', border_mode='same')(inputs)
    c2_1 = actv()(c2_1)
    if splitted:
        c2_2 = Convolution2D(int(depth/2), 1, 3, init='he_normal', border_mode='same')(c2_1)
        c2_2 = BatchNormalization(axis=3)(c2_2)
        c2_2 = actv()(c2_2)
        c2_3 = Convolution2D(int(depth/2), 3, 1, init='he_normal', border_mode='same')(c2_2)
    else:
        c2_3 = Convolution2D(int(depth/2), 3, 3, init='he_normal', border_mode='same')(c2_1)
    
    c3_1 = Convolution2D(int(depth/16), 1, 1, init='he_normal', border_mode='same')(inputs)
    #missed batch norm
    c3_1 = actv()(c3_1)
    if splitted:
        c3_2 = Convolution2D(int(depth/8), 1, 5, init='he_normal', border_mode='same')(c3_1)
        c3_2 = BatchNormalization(axis=3)(c3_2)
        c3_2 = actv()(c3_2)
        c3_3 = Convolution2D(int(depth/8), 5, 1, init='he_normal', border_mode='same')(c3_2)
    else:
        c3_3 = Convolution2D(int(depth/8), 5, 5, init='he_normal', border_mode='same')(c3_1)
    
    p4_1 = MaxPooling2D(pool_size=(3,3), strides=(1,1), border_mode='same')(inputs)
    c4_2 = Convolution2D(int(depth/8), 1, 1, init='he_normal', border_mode='same')(p4_1)
    
    res = merge([c1_1, c2_3, c3_3, c4_2], mode='concat', concat_axis=3)
    res = BatchNormalization(axis=3)(res)
    res = actv()(res)
    return res
    

def rblock(inputs, num, depth, scale=0.1):    
    residual = Convolution2D(depth, num, num, border_mode='same')(inputs)
    residual = BatchNormalization(axis=3)(residual)
    residual = Lambda(lambda x: x*scale)(residual)
    res = _shortcut(inputs, residual)
    return ELU()(res) 
    

def NConvolution2D(nb_filter, nb_row, nb_col, border_mode='same', subsample=(1, 1)):
    def f(_input):
        conv = Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample,
                              border_mode=border_mode)(_input)
        norm = BatchNormalization(axis=3)(conv)
        return ELU()(norm)

    return f

def BNA(_input):
    inputs_norm = BatchNormalization(axis=3)(_input)
    return ELU()(inputs_norm)

def reduction_a(inputs, k=64, l=64, m=96, n=96):
    "35x35 -> 17x17"
    inputs_norm = BNA(inputs)
    pool1 = MaxPooling2D((3,3), strides=(2,2), border_mode='same')(inputs_norm)
    
    conv2 = Convolution2D(n, 3, 3, subsample=(2,2), border_mode='same')(inputs_norm)
    
    conv3_1 = NConvolution2D(k, 1, 1, subsample=(1,1), border_mode='same')(inputs_norm)
    conv3_2 = NConvolution2D(l, 3, 3, subsample=(1,1), border_mode='same')(conv3_1)
    conv3_2 = Convolution2D(m, 3, 3, subsample=(2,2), border_mode='same')(conv3_2)
    
    res = merge([pool1, conv2, conv3_2], mode='concat', concat_axis=3)
    return res


def reduction_b(inputs):
    "17x17 -> 8x8"
    inputs_norm = BNA(inputs)
    pool1 = MaxPooling2D((3,3), strides=(2,2), border_mode='same')(inputs_norm)
    #
    conv2_1 = NConvolution2D(64, 1, 1, subsample=(1,1), border_mode='same')(inputs_norm)
    conv2_2 = Convolution2D(96, 3, 3, subsample=(2,2), border_mode='same')(conv2_1)
    #
    conv3_1 = NConvolution2D(64, 1, 1, subsample=(1,1), border_mode='same')(inputs_norm)
    conv3_2 = Convolution2D(72, 3, 3, subsample=(2,2), border_mode='same')(conv3_1)
    #
    conv4_1 = NConvolution2D(64, 1, 1, subsample=(1,1), border_mode='same')(inputs_norm)
    conv4_2 = NConvolution2D(72, 3, 3, subsample=(1,1), border_mode='same')(conv4_1)
    conv4_3 = Convolution2D(80, 3, 3, subsample=(2,2), border_mode='same')(conv4_2)
    #
    res = merge([pool1, conv2_2, conv3_2, conv4_3], mode='concat', concat_axis=3)
    return res
    
    


def get_unet_inception_2head(optimizer):
    splitted = True
    act = 'elu'
    
    inputs = Input((IMG_ROWS, IMG_COLS, 3), name='main_input')
    conv1 = inception_block(inputs, 32, batch_mode=2, splitted=splitted, activation=act)
    #conv1 = inception_block(conv1, 32, batch_mode=2, splitted=splitted, activation=act)
    
    #pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = NConvolution2D(32, 3, 3, border_mode='same', subsample=(2,2))(conv1)
    pool1 = Dropout(0.5)(pool1)
    
    conv2 = inception_block(pool1, 64, batch_mode=2, splitted=splitted, activation=act)
    #pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = NConvolution2D(64, 3, 3, border_mode='same', subsample=(2,2))(conv2)
    pool2 = Dropout(0.5)(pool2)
    
    conv3 = inception_block(pool2, 128, batch_mode=2, splitted=splitted, activation=act)
    #pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = NConvolution2D(128, 3, 3, border_mode='same', subsample=(2,2))(conv3)
    pool3 = Dropout(0.5)(pool3)
     
    conv4 = inception_block(pool3, 256, batch_mode=2, splitted=splitted, activation=act)
    #pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = NConvolution2D(256, 3, 3, border_mode='same', subsample=(2,2))(conv4)
    pool4 = Dropout(0.5)(pool4)
    
    conv5 = inception_block(pool4, 512, batch_mode=2, splitted=splitted, activation=act)
    #conv5 = inception_block(conv5, 512, batch_mode=2, splitted=splitted, activation=act)
    conv5 = Dropout(0.5)(conv5)
    
    after_conv4 = rblock(conv4, 1, 256)
    up6 = merge([UpSampling2D(size=(2, 2))(conv5), after_conv4], mode='concat', concat_axis=3)
    conv6 = inception_block(up6, 256, batch_mode=2, splitted=splitted, activation=act)
    conv6 = Dropout(0.5)(conv6)
    
    after_conv3 = rblock(conv3, 1, 128)
    up7 = merge([UpSampling2D(size=(2, 2))(conv6), after_conv3], mode='concat', concat_axis=3)
    conv7 = inception_block(up7, 128, batch_mode=2, splitted=splitted, activation=act)
    conv7 = Dropout(0.5)(conv7)
    
    after_conv2 = rblock(conv2, 1, 64)
    up8 = merge([UpSampling2D(size=(2, 2))(conv7), after_conv2], mode='concat', concat_axis=3)
    conv8 = inception_block(up8, 64, batch_mode=2, splitted=splitted, activation=act)
    conv8 = Dropout(0.5)(conv8)
    
    after_conv1 = rblock(conv1, 1, 32)
    up9 = merge([UpSampling2D(size=(2, 2))(conv8), after_conv1], mode='concat', concat_axis=3)
    conv9 = inception_block(up9, 32, batch_mode=2, splitted=splitted, activation=act)
    #conv9 = inception_block(conv9, 32, batch_mode=2, splitted=splitted, activation=act)
    conv9 = Dropout(0.5)(conv9)

    conv10 = Convolution2D(1, 1, 1, init='he_normal', activation='sigmoid', name='main_output')(conv9)
    #print conv10._keras_shape

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=optimizer,
                  loss=bce_dice_loss,
                  metrics=[dice_coeff])

    return model


get_unet = get_unet_inception_2head

In [4]:
model = get_unet(Adam(lr=1e-5))

/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (1, 1), padding="same", kernel_initializer="he_normal")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (1, 1), padding="same", kernel_initializer="he_normal")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 3), padding="same", kernel_initializer="he_normal")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 1), padding="same", kernel_initializer="he_normal")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 1), padding="same", kerne

/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), padding="same")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:149: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), padding="same")`
/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use inste

# Modeling

In [5]:
input_size = 128
max_epochs = 50
train_batch_size = 16
val_batch_size=32
orig_width = 1918
orig_height= 1280
threshold  = 0.5

In [6]:
df_train = pd.read_csv('data/train_masks.csv')

In [7]:
df_train.head()

,img,rle_mask
0,00087a6bd4dc_01.jpg,879386 40 881253 141 883140 205 885009 17 8850...
1,00087a6bd4dc_02.jpg,873779 4 875695 7 877612 9 879528 12 881267 15...
2,00087a6bd4dc_03.jpg,864300 9 866217 13 868134 15 870051 16 871969 ...
3,00087a6bd4dc_04.jpg,879735 20 881650 26 883315 92 883564 30 885208...
4,00087a6bd4dc_05.jpg,883365 74 883638 28 885262 119 885550 34 88716...


In [8]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [9]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [26]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [27]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [28]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [13]:
all_imgs  = {}
all_masks = {}
for id in ids_train:
    img  = cv2.imread('data/train/{}.jpg'.format(id))
    img  = cv2.resize(img, (input_size, input_size))
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (input_size, input_size))
    all_imgs[id]  = img
    all_masks[id] = mask

In [30]:
import random

In [31]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(list(ids_train_split), len(list(ids_train_split)))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch


In [32]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch.values:
                img  = all_imgs[id]
                mask = all_masks[id]
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
callbacks = [EarlyStopping(monitor='val_dice_coeff',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_coeff',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_coeff',
                             filepath='weights/unet-inception-128.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
208s - loss: 1.2588 - dice_coeff: 0.4351 - val_loss: 1.1301 - val_dice_coeff: 0.3091
Epoch 2/50
192s - loss: 0.9890 - dice_coeff: 0.5376 - val_loss: 1.1081 - val_dice_coeff: 0.3145
Epoch 3/50
192s - loss: 0.8052 - dice_coeff: 0.6017 - val_loss: 0.8988 - val_dice_coeff: 0.4504
Epoch 4/50
192s - loss: 0.6790 - dice_coeff: 0.6488 - val_loss: 0.8308 - val_dice_coeff: 0.4970
Epoch 5/50
192s - loss: 0.5943 - dice_coeff: 0.6836 - val_loss: 0.7180 - val_dice_coeff: 0.5706
Epoch 6/50
192s - loss: 0.5322 - dice_coeff: 0.7107 - val_loss: 0.6272 - val_dice_coeff: 0.6281
Epoch 7/50
192s - loss: 0.4846 - dice_coeff: 0.7328 - val_loss: 0.5708 - val_dice_coeff: 0.6643
Epoch 8/50
192s - loss: 0.4419 - dice_coeff: 0.7536 - val_loss: 0.5256 - val_dice_coeff: 0.6938
Epoch 9/50
192s - loss: 0.4078 - dice_coeff: 0.7710 - val_loss: 0.4832 - val_dice_coeff: 0.7208
Epoch 10/50
192s - loss: 0.3791 - dice_coeff: 0.7851 - val_loss: 0.4303 - val_dice_coeff: 0.7525
Epoch 11/50
192s - loss: 0.3553 - dice_

In [17]:
model.load_weights('./weights/unet-inception-128.hdf5')

In [18]:
callbacks = [EarlyStopping(monitor='val_dice_coeff',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_coeff',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_coeff',
                             filepath='weights/unet-inception-128.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
139s - loss: 0.1539 - dice_coeff: 0.9131 - val_loss: 0.2070 - val_dice_coeff: 0.8934
Epoch 2/50
131s - loss: 0.1496 - dice_coeff: 0.9156 - val_loss: 0.2055 - val_dice_coeff: 0.8945
Epoch 3/50
132s - loss: 0.1458 - dice_coeff: 0.9181 - val_loss: 0.1965 - val_dice_coeff: 0.8996
Epoch 4/50
132s - loss: 0.1418 - dice_coeff: 0.9204 - val_loss: 0.1937 - val_dice_coeff: 0.9013
Epoch 5/50
132s - loss: 0.1387 - dice_coeff: 0.9223 - val_loss: 0.1989 - val_dice_coeff: 0.8996
Epoch 6/50
132s - loss: 0.1360 - dice_coeff: 0.9243 - val_loss: 0.1869 - val_dice_coeff: 0.9056
Epoch 7/50
132s - loss: 0.1320 - dice_coeff: 0.9265 - val_loss: 0.1746 - val_dice_coeff: 0.9115
Epoch 8/50
132s - loss: 0.1296 - dice_coeff: 0.9282 - val_loss: 0.1777 - val_dice_coeff: 0.9107
Epoch 9/50
132s - loss: 0.1265 - dice_coeff: 0.9299 - val_loss: 0.1771 - val_dice_coeff: 0.9114
Epoch 10/50
132s - loss: 0.1233 - dice_coeff: 0.9318 - val_loss: 0.1671 - val_dice_coeff: 0.9164
Epoch 11/50
132s - loss: 0.1218 - dice_

In [19]:
history.history

{'dice_coeff': [0.91305642520473396,
  0.91554450253596764,
  0.91804631673147408,
  0.92046436806568643,
  0.92232778283243388,
  0.92428327968723945,
  0.92652074072226265,
  0.92815641823799078,
  0.92984956218803838,
  0.93176751886597431,
  0.93295332664939634,
  0.93453360623165194,
  0.93589913030807159,
  0.93733690063256303,
  0.93836108657007433,
  0.9397426873518735,
  0.94075169097586409,
  0.94171554523838241,
  0.94288925254667122,
  0.94380051820225441,
  0.94473870313138286,
  0.94570001773810797,
  0.94670223026955158,
  0.94737135631739366,
  0.9482602834408641,
  0.94918990864507691,
  0.94993915809753193,
  0.95077761036758046,
  0.95121461759328252,
  0.95185924522413956,
  0.9527948460063419,
  0.95322414967586133,
  0.95365553449940033,
  0.95463427287647706,
  0.95467898125144712,
  0.95487894534478901,
  0.95488508465249067,
  0.95490348257069502],
 'loss': [0.15395652139216148,
  0.14959642302784931,
  0.14577361646975581,
  0.14180872548710216,
  0.1387315109

# v2

In [3]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
    """Utility function to apply conv + BN.

    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.

    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

In [4]:
def inception_block(x, nf):
    branch1x1 = conv2d_bn(x, int(nf/4), 1, 1)
    
    branch5x5 = conv2d_bn(x, int(nf*3/16), 1, 1)
    branch5x5 = conv2d_bn(branch5x5, int(nf/4), 5, 5)

    branch3x3dbl = conv2d_bn(x, int(nf/4), 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, int(nf*3/8), 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, int(nf*3/8), 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, int(nf/8), 1, 1)

    out = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],axis=3)
    return out

In [5]:
inp = Input(shape=(256,256,3))

In [6]:
down1 = inception_block(inp, 32)
down1 = inception_block(down1, 32)
down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)

In [7]:
down2 = inception_block(down1_pool, 64)
down2 = inception_block(down2, 64)
down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)

In [8]:
down3 = inception_block(down2_pool, 128)
down3 = inception_block(down3, 128)
down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)

In [9]:
down4 = inception_block(down3_pool, 256)
down4 = inception_block(down4, 256)
down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)

In [10]:
down5 = inception_block(down4_pool, 512)
down5 = inception_block(down5, 512)
down5_pool = MaxPooling2D((2, 2), strides=(2, 2))(down5)

In [11]:
center = inception_block(down5_pool, 1024)
center = inception_block(center, 1024)

In [12]:
up5 = UpSampling2D((2, 2))(center)
up5 = concatenate([down5, up5], axis=3)
up5 = inception_block(up5, 512)
up5 = inception_block(up5, 512)

In [13]:
up4 = UpSampling2D((2, 2))(up5)
up4 = concatenate([down4, up4], axis=3)
up4 = inception_block(up4, 256)
up4 = inception_block(up4, 256)

In [14]:
up3 = UpSampling2D((2, 2))(up4)
up3 = concatenate([down3, up3], axis=3)
up3 = inception_block(up3, 128)
up3 = inception_block(up3, 128)

In [15]:
up2 = UpSampling2D((2, 2))(up3)
up2 = concatenate([down2, up2], axis=3)
up2 = inception_block(up2, 64)
up2 = inception_block(up2, 64)

In [16]:
up1 = UpSampling2D((2, 2))(up2)
up1 = concatenate([down1, up1], axis=3)
up1 = inception_block(up1, 32)
up1 = inception_block(up1, 32)

In [17]:
classify = Conv2D(1, (1, 1), activation='sigmoid')(up1)

In [18]:
model = Model(inputs=inp, outputs=classify)

In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
conv2d_4 (Conv2D)                (None, 256, 256, 8)   24          input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, 256, 256, 8)   24          conv2d_4[0][0]                   
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 256, 256, 8)   0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

In [20]:
from keras.utils import plot_model  
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)  
from IPython.display import FileLink
FileLink('model.png')

/home/sainath/extvol/kaggle-carvana-image-masking-challenge/model.png

In [22]:
input_size = 256
max_epochs = 50
train_batch_size = 16
val_batch_size=32
orig_width = 1918
orig_height= 1280
threshold  = 0.5

In [23]:
df_train = pd.read_csv('data/train_masks.csv')

In [24]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [25]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [29]:
all_imgs  = {}
all_masks = {}
for id in ids_train:
    img  = cv2.imread('data/train/{}.jpg'.format(id))
    img  = cv2.resize(img, (input_size, input_size))
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (input_size, input_size))
    all_imgs[id]  = img
    all_masks[id] = mask

In [34]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
callbacks = [EarlyStopping(monitor='val_dice_coeff',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_coeff',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_coeff',
                             filepath='weights/unet-inception-256.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
1193s - loss: 0.6007 - dice_coeff: 0.6648 - val_loss: 1.3479 - val_dice_coeff: 0.1840
Epoch 2/50
1138s - loss: 0.2524 - dice_coeff: 0.8392 - val_loss: 0.2122 - val_dice_coeff: 0.8652
Epoch 4/50
1138s - loss: 0.1893 - dice_coeff: 0.8781 - val_loss: 0.1624 - val_dice_coeff: 0.8956
Epoch 5/50
1138s - loss: 0.1422 - dice_coeff: 0.9082 - val_loss: 0.1286 - val_dice_coeff: 0.9208
Epoch 6/50
1138s - loss: 0.1072 - dice_coeff: 0.9312 - val_loss: 0.0922 - val_dice_coeff: 0.9409
Epoch 7/50
1138s - loss: 0.0818 - dice_coeff: 0.9481 - val_loss: 0.0679 - val_dice_coeff: 0.9572
Epoch 8/50
1138s - loss: 0.0631 - dice_coeff: 0.9607 - val_loss: 0.0547 - val_dice_coeff: 0.9667
Epoch 9/50
1138s - loss: 0.0499 - dice_coeff: 0.9697 - val_loss: 0.0442 - val_dice_coeff: 0.9737
Epoch 10/50
1138s - loss: 0.0409 - dice_coeff: 0.9760 - val_loss: 0.0348 - val_dice_coeff: 0.9796
Epoch 11/50
1138s - loss: 0.0346 - dice_coeff: 0.9803 - val_loss: 0.0293 - val_dice_coeff: 0.9834
Epoch 12/50
1138s - loss: 0.

KeyboardInterrupt: 